In [ ]:
from pathlib import Path

import SAGA
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from easyquery import *
from SAGA.utils import *
from SAGA.utils.display import show_images
from astropy.table import vstack, Table, join, unique
from astropy.coordinates import Distance, SkyCoord
from SAGA.objects.build2 import add_surface_brightness

import numexpr as ne
from scipy.stats import binned_statistic
#from bottleneck import nanmax, nanmin
from itertools import product, chain

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)

assert len(np.unique(base["HOSTID"])) == 36

In [ ]:
specs_full = Query(C.basic_cut2, C.has_spec, "RHOST_KPC > 10", "SPEC_Z >= 0.002").filter(base)

In [ ]:
assert not Query("SPEC_Z < 0.015", ~C.paper2_targeting_cut).mask(specs_full).any()

In [ ]:
hosts = np.unique(specs_full["HOSTID"])
host_dist = saga.host_catalog.load(hosts)["DIST"]

full_sq_deg = (np.rad2deg(np.arcsin(0.3 / host_dist))**2 - np.rad2deg(np.arcsin(0.01 / host_dist))**2).sum() * np.pi
print(full_sq_deg)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,3.6), gridspec_kw={"width_ratios": (3,2), "wspace": 0.27})

ax_inset = fig.add_axes([0.34, 0.27, 0.18, 0.34])

for ax_this, zbins in zip([ax[0], ax_inset], [np.linspace(0, 1, 41), np.linspace(0, 0.05, 11)]):
    zbins[0] = 0.001
    for q, kw in (
        (Query(), dict(label='All redshifts', color=plt.cm.Oranges(0.25))),
        (C.gri_cut, dict(label='Paper I $gri$ cuts', color=plt.cm.Blues(0.35))),
        (C.paper2_targeting_cut, dict(label='Primary targets (this work)', color=plt.cm.Greens(0.65))),
    ):
        ax_this.hist(q.filter(specs_full, 'SPEC_Z'), zbins, weights=np.ones(q.count(specs_full))/full_sq_deg, **kw)

        ax_this.set_xlim(0, 1 if zbins[-1] == 1 else 0.052)

ax_inset.text(0.003, 5.8, "zoom-in")
ax[0].set_ylabel(r'$N_{\rm spec}$ (per sq. deg., per $z$ bin)')
ax[0].set_xlabel(r'$z_{\rm spec}$')
ax[0].set_title("$N(z)$ split by targeting regions", fontsize=14)

ax_inset.axvline(0.003, ls=':', lw=1, color="grey")
ax_inset.axvline(0.013, ls=':', lw=1, color="grey")
ax_this = ax[-1]
zbins = np.linspace(0, 0.8, 33)
zbins[0] = 0.001
faint = Query("r_mag >= 19")
plt.rcParams['hatch.linewidth'] = 0.25

for q, kw in (
    (Query(), dict(color=plt.cm.YlGn(0.27), hatch="////")),
    (Query(C.has_our_specs_only | faint) , dict(label='From literature', color=plt.cm.YlGn(0.2))),
    (Query(C.has_our_specs_only) , dict(color=plt.cm.Purples(0.57), hatch="////", )),
    (Query(C.has_our_specs_only, faint) , dict(label='From SAGA', color=plt.cm.Purples(0.5))),
):
    ax_this.hist(q.filter(specs_full, 'SPEC_Z'), zbins, weights=np.ones(q.count(specs_full))/full_sq_deg, **kw)

ax_this.set_xlabel(r'$z_{\rm spec}$')
ax_this.set_ylabel(r'$N_{\rm spec}$ (per sq. deg., per $z$ bin)')
ax_this.set_xlim(0, 0.8)
        
ax_this.set_title("$N(z)$ split by source and magnitude", fontsize=14)
ax_this.text(0.75, 50, "Hatched: $r_o < 19$", ha="right")

ax_this.legend(loc='upper right', markerfirst=False, handletextpad=0.2, fontsize=13)

ax[0].legend(loc='upper right', markerfirst=False, handletextpad=0.2, fontsize=13)
plt.savefig('/home/yymao/Downloads/z_distribution.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(9,4))

for ax_this, zbins in zip(ax, [np.linspace(0, 0.8, 41), np.linspace(0, 0.05, 11)]):
    zbins[0] = 0.001
    for q, kw in (
        (Query(), dict(label='All redshifts', color=plt.cm.Oranges(0.25))),
        (C.gri_cut, dict(label='Paper I $gri$ cuts', color=plt.cm.Blues(0.35))),
        (C.paper2_targeting_cut, dict(label='This work', color=plt.cm.Greens(0.65))),
    ):
        ax_this.hist(q.filter(specs_full, 'SPEC_Z'), zbins, weights=np.ones(q.count(specs_full))/full_sq_deg, **kw)
        ax_this.set_xlim(0, None)

ax[1].set_title("$N(z)$, zoom-in", fontsize=14)
ax[0].set_ylabel(r'$N_{\rm spec}$ (per sq. deg., per $z$ bin)')
ax[0].set_xlabel(r'$z_{\rm spec}$')
ax[1].set_xlabel(r'$z_{\rm spec}$')
ax[0].set_title("$N(z)$ split by targeting regions", fontsize=14)


ax[1].axvline(0.003, ls=':', lw=1, color="grey")
ax[1].axvline(0.013, ls=':', lw=1, color="grey")
ax[0].legend(loc='upper right', markerfirst=False, handletextpad=0.2, fontsize=13)
plt.tight_layout()


## appendix plots 

In [ ]:
base1 = saga.host_catalog.construct_host_query("COVERAGE_SDSS >= 0.99").filter(base)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(10, 4))
for i, ax_this in enumerate(ax):
    
    if i == 0:
        key = "r_mag"
        ax_this.set_xlabel(r'$r_o$')
        bins = np.linspace(11, 21, 21)
        bins[-1] = 20.75
    elif i == 1:
        key = "sb_r"
        ax_this.set_xlabel(r'$\mu_{r_o,{\rm eff}}$')
        bins = np.linspace(18, 26, 17)
        cut_other = Query(C.faint_end_limit, C.paper2_targeting_cut)
    elif i == 2:
        key = "gr"
        ax_this.set_xlabel(r'$(g-r)_o$')
        bins = np.linspace(-0.25, 1.25, 19)
        cut_other = Query(C.faint_end_limit, C.paper2_targeting_cut)
        
    bin_index = np.digitize(base1[key], bins)
    cut = Query(C.faint_end_limit, C.paper2_targeting_cut)
    
    x = bins[:-1]
    width = np.ediff1d(bins)
    def add_bar(*queries, **kwargs):
        y = np.maximum(np.bincount(bin_index[Query(*queries).mask(base1)], minlength=len(bins)+1)[1:-1], 0.01) 
        return ax_this.bar(x, y, width=width, align='edge', **kwargs)

    c_value = 0.35
    l0 = add_bar(color=plt.cm.Greys(c_value))
    __ = add_bar(C.has_spec, color=plt.cm.Blues(c_value))
    __ = add_bar(C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    
    if False:
        c_value = 0.5
        __ = add_bar(cut_other, color=plt.cm.Greys(c_value))
        __ = add_bar(cut_other, C.has_spec, color=plt.cm.Blues(c_value))
        __ = add_bar(cut_other, C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    
    c_value = 0.65
    l1 = add_bar(cut, color=plt.cm.Greys(c_value))
    l2 = add_bar(cut, C.has_spec, color=plt.cm.Blues(c_value))
    l3 = add_bar(cut, C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    l4 = add_bar(cut, C.has_spec, C.is_sat, color=plt.cm.Oranges(c_value))
    
    ax_this.set_yscale("symlog", linthresh=10, subs=[2,3,4,5,6,7,8,9], linscale=np.log10(2))
    ax_this.set_ylim(0, 3e4)
    ax_this.grid("on", axis="y", color="k", lw=0.5)
    ax_this.set_xlim(bins[0], bins[-1])
    if i == 0:
        bins[-1] = 21
    #ax_this.set_xticks(bins[::5] if i == 2 else bins[::4])
    

ax[0].set_ylabel(r'$N_{\rm galaxy}$ (per bin)')

x0, y0 = 12, 2e3
for mult in (1.1, 2, 5):
    ax[0].plot([x0, x0], [y0, y0*mult], c='k', lw=2, marker='')
    ax[0].text(x0, y0*0.88, fr'$\times\,{mult}$', ha="center", va="top")
    x0 += 1.8

plt.tight_layout(rect=(0,0,1,0.9), w_pad=0.1)

plt.figlegend((l1, l0), ("Within primary targeting region", "Outside of primary targeting region"), loc="upper right", markerfirst=False, bbox_to_anchor=(0.985,1.02))
plt.figlegend((l1, l2, l3, l4), ("No redshift", "Pre-existing redshifts", "SAGA redshifts", "Satellites"), loc="upper left", ncol=2, bbox_to_anchor=(0.08,1.02))

plt.savefig('/home/yymao/Downloads/target_spec_counts.pdf', bbox_inches='tight')